# Extracting data from DICOM files
### This notebook extract data from Dicom files using the dicomTags.

#### Installing required packages:

1. Create a new environment (https://docs.python.org/3/library/venv.html)

```python
python3 -m venv /path/to/new/virtual/environment
```
2. Activate the new environment

```python
source env/bin/activate
```

3. Install required packages

```python
pip install -r requirements.txt
```

4. Run the notebook.  :)

In [1]:
import pydicom
import os
import glob
from tqdm.notebook import tqdm
import csv

## Find data
Download data from https://wiki.cancerimagingarchive.net/display/Public/Soft-tissue-Sarcoma (~10GB)  
and look if they are present in folder "data"

In [2]:

DICOM_files = glob.glob('exampleRadiotherapy/**/*.[dD][cC][mM]', recursive=True)

## Print some dicom file paths

In [3]:
print(DICOM_files[:10])

['exampleRadiotherapy/GB32/RTDOSE/RT_THORAX(Adulte)_/20220930/162529.017954_/(0018,0050)_DS_(no_value)_________________________________________#_0,1_Slice_Thickness/2.16.840.1.114337.1.8672.1673611911.0.1.3/2.16.840.1.114337.1.8672.1673611915.0.dcm', 'exampleRadiotherapy/GB32/RTDOSE/RT_THORAX(Adulte)_/20220930/162529.017954_/(0018,0050)_DS_(no_value)_________________________________________#_0,1_Slice_Thickness/2.16.840.1.114337.1.8672.1673611911.0.1.3/2.16.840.1.114337.1.8672.1673611912.1.dcm', 'exampleRadiotherapy/GB32/RTDOSE/RT_ORL(Adulte)/20221024/154716.999045_/(0018,0050)_DS_(no_value)_________________________________________#_0,1_Slice_Thickness/2.16.840.1.114337.1.8672.1673611989.0.1.3/2.16.840.1.114337.1.8672.1673611992.0.dcm', 'exampleRadiotherapy/GB32/RTDOSE/RT_ORL(Adulte)/20221024/154716.999045_/(0018,0050)_DS_(no_value)_________________________________________#_0,1_Slice_Thickness/2.16.840.1.114337.1.8672.1673611989.0.1.3/2.16.840.1.114337.1.8672.1673611989.2.dcm', 'exampl

### Create the dictionaries containing the dicom name and dicom tag from the pivot csv files  
from https://docs.google.com/spreadsheets/d/1doEWSp2v92FaIbbFMqj4UZKJOYILcjoL/edit#gid=400416101


In [4]:
dictTagAnalysis = { }

dictTagStudy = {
                "Patient_Id": [('0x0010', '0x0020')],
                "Study_StudyInstanceUID": [('0x0020', '0x000d')],
                "Study_StudyDescription": [('0x0008', '0x1030')],
                "Study_InstitutionName": [('0x0008', '0x0080')],
                "Study_ModalitiesInStudy": [('0x0008', '0x0061')],
                "Study_NbStudyRelatedSeries": [('0x0020', '0x1206')],
                "Study_AcquisitionDate": [('0x0008', '0x0022'), ('0x0008', '0x0032')],
                }

dictTagSeries = {
                "Patient_Id": [('0x0010', '0x0020')],
                "Series_SeriesNumber": [('0x0020', '0x0011')],
                "Series_SeriesInstanceUID": [('0x0020', '0x000e')],
                "Series_Modality": [('0x0008', '0x0060')],
                "Series_AcquisitionDate": [('0x0008', '0x0022'), ('0x0008', '0x0032')],
                "Series_Description": [('0x0008', '0x103e'), ('0x0008', '0x1030')],
                "Series_BodyPartExamined": [('0x0018', '0x0015')],
                "Series_NbSeriesRelatedInstances": [('0x0020', '0x1209')],
                "Series_PatientWeight": [('0x0010', '0x1030')],
                "Series_PatientSize": [('0x0010', '0x1020')],
                }


dictTagEquipment = {
                "Patient_Id": [('0x0010', '0x0020')],
                "Equipment_ModelName": [('0x0008', '0x1090')],
                "Equipment_Manufacturer": [('0x0008', '0x0070')],
                "Equipment_SoftwareVersion": [('0x0018', '0x1020')],
                   }

dictTagInjection = {
                "Patient_Id": [('0x0010', '0x0020')],
                "Injection_Radiopharmaceutical": [('0x0018', '0x0031')],
                "Injection_ContrastBolusAgent": [('0x0018', '0x0010')],
                "Injection_ContrastBolusStartTime": [('0x0018', '0x1042')],
                "Injection_ContrastBolusStopTime": [('0x0018', '0x1043')],
                "Injection_RadiopharmaceuticalStartTime": [('0x0018', '0x1072')],
                "Injection_RadionuclideTotalDose": [('0x0018', '0x1074')],
                   }

dictTagROISegmentation = {
                "Patient_Id": [('0x0010', '0x0020')],
                "ROISegmentation_ReferencedSOPInstanceUID": [('0x0008', '0x1155')],
                "ROISegmentation_ROINumber": [('0x3006', '0x0084')],
                "ROISegmentation_ROIName": [('0x3006', '0x0026')],
                "ROISegmentation_ROIDescription": [('0x3006', '0x0028')],
                "ROISegmentation_ROIType": [],
                "ROISegmentation_ROIFilename": [],
                         }

dictTagCommonImage = {
                "Patient_Id": [('0x0010', '0x0020')],
                "CommonImage_SOPInstanceUID": [('0x0008', '0x0018')],
                "CommonImage_SliceThickness": [('0x0018', '0x0050')],
                "CommonImage_PixelSpacing": [('0x0028', '0x0030')],
                "CommonImage_FieldOfView": [],
                "CommonImage_Rows": [('0x0028', '0x0010')],
                "CommonImage_Columns": [('0x0028', '0x0011')],
                     }

dictTagCTImage = {
                "Patient_Id": [('0x0010', '0x0020')],
                "CTImage_KVp": [('0x0018', '0x0060')],
                "CTImage_XRayTubeCurrent": [('0x0018', '0x1151')],
                "CTImage_ExposureTime": [('0x0018', '0x1150')],
                "CTImage_SpiralPitchFactor": [('0x0018', '0x9311')],
                "CTImage_FilterType": [('0x0018', '0x1160')],
                "CTImage_ConvolutionKernel": [('0x0018', '0x1210')],
                 }

dictTagDXImage = {
                "Patient_Id": [('0x0010', '0x0020')],
                "DXImage_ImageLaterality": [('0x0020', '0x0062')],
                "DXImage_PatientOrientation": [('0x0020', '0x0020')],
                "DXImage_KVP": [('0x0008', '0x0060')],
                "DXImage_Exposure": [('0x0018', '0x1152')],
                "DXImage_ExposureTime": [('0x0018', '0x1150')],
                 }

dictTagMRImage = {
                "Patient_Id": [('0x0010', '0x0020')],
                "MRImage_SequenceName": [('0x0018', '0x0024')],
                "MRImage_MagneticFieldStrength": [('0x0018', '0x0087')],
                "MRImage_MRAcquisitionType": [('0x0018', '0x0023')],
                "MRImage_RepetitionTime": [('0x0018', '0x0080')],
                "MRImage_EchoTime": [('0x0018', '0x0081')],
                "MRImage_ImagingFrequency": [('0x0018', '0x0084')],
                "MRImage_FlipAngle": [('0x0018', '0x0024')],
                "MRImage_InversionTime": [('0x0018', '0x0082')],
                "MRImage_ReceiveCoilName": [('0x0018', '0x1250')],
                 }

dictTagNMImage = {
                "Patient_Id": [('0x0010', '0x0020')],
                "NMImage_AttenuationCorrectionMethod": [('0x0054', '0x1101')],
                "NMImage_ReconstructionMethod": [('0x0054', '0x1103')],
                "NMImage_ScatterCorrectionMethod": [('0x0054', '0x1105')],
                 }





## Define some usefull class and functions
The class Pivot represent a pivot csv file

In [5]:

# Return the concatenation of all tag values from DICOM_file
# look inside sequences
def get_tag_value(DICOM_file, tags, nbStruct=-1):
    value = ""
    for tag in tags:
        try:
            value += " " + str(DICOM_file[tag[0], tag[1]].value)
        except:
            for elem in DICOM_file:
                if elem.VR == 'SQ':
                    if nbStruct >= 0 and len(elem.value) > nbStruct and pydicom.tag.Tag(tag[0], tag[1]) in elem.value[nbStruct]:
                        tmp_value = get_tag_value(elem.value[nbStruct], [tag])
                        if tmp_value != "UMLS:C0439673":
                            value += " " + tmp_value
                    else:
                        for item in elem.value:
                            tmp_value = get_tag_value(item, [tag])
                            if tmp_value != "UMLS:C0439673":
                                value += " " + tmp_value
    if value.strip() == "":
        value = "UMLS:C0439673"
    return(value.strip())

class Pivot:
    # Create a pivot element with:
    #  - a dictionary containing all the entries (= aline in the .csv file). The key is self.key
    #  - instanceId is an int = line of the .csv file
    #  - key is the key of the self.dict
    #  - dictTag is a dictionay containing the Header of the .csv files and the corresponding dicom tags
    #  - columns is an array containing the header of the csv file and starts with Patient_Id and Instance_Id
    def __init__(self, key, dictTag):
        self.dict = {}
        self.instanceId = 0
        self.key = key
        self.dictTag = dictTag
        self.columns = ["Patient_Id", "Instance_Id"] + list(self.dictTag.keys())[1:]

    # Return the list of element already present in the dict
    def elements(self):
        return(list(self.dict.keys()))
    
    # Write the pivot as .csv
    def write(self, output):
        with open(output, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=self.columns, lineterminator='\n')
            writer.writeheader()
            for el in self.dict.keys():
                writer.writerow(self.dict[el])

class PivotDicom(Pivot):
    def __init__(self, key, dictTag):
        Pivot.__init__(self, key, dictTag)

    # Fill the dict with the current DICOM_file (= add a line in the .csv file) from dicom
    def fill(self, DICOM_file, nbStruct=-1):
        key = DICOM_file[self.key].value + "_" + str(nbStruct)
        self.dict[key] = {}
        for el in list(self.dictTag.keys()):
            self.dict[key][el] = get_tag_value(DICOM_file, self.dictTag[el], nbStruct)
        self.dict[key]["Instance_Id"] = self.instanceId
        self.instanceId += 1

    # Fill the dict with the current DICOM_file (= add a line in the .csv file) from a dict
    def fillFromDict(self, dictionary, nbStruct=-1):
        key = dictionary["ROISegmentation_ROIFilename"] + "_" + str(nbStruct)
        self.dict[key] = {}
        for el in list(self.dictTag.keys()):
                self.dict[key][el] = dictionary[el]
        self.dict[key]["Instance_Id"] = self.instanceId
        self.instanceId += 1

    # Add reference from another pivot and add the entry after Patient_Id and Instance_Id in the header if not present
    def addRef(self, DICOM_file, name, pivot, nbStruct=-1):
        key = DICOM_file[self.key].value + "_" + str(nbStruct)
        self.dict[key][name] = pivot.dict[DICOM_file[pivot.key].value + "_-1"]["Instance_Id"]
        if name not in self.columns:
            self.columns = self.columns[0:2] + [name] + self.columns[2:]

    # Add reference from another pivot and add the entry after Patient_Id and Instance_Id in the header if not present
    def addRefFromDict(self, dictionary, name, pivot, nbStruct=-1):
        key = dictionary["ROISegmentation_ROIFilename"] + "_" + str(nbStruct)
        self.dict[key][name] = pivot.dict[dictionary["Series_Ref"] + "_-1"]["Instance_Id"]
        if name not in self.columns:
            self.columns = self.columns[0:2] + [name] + self.columns[2:]

class PivotRadiomics(Pivot):
    def __init__(self):
        Pivot.__init__(self, '', {})
    
    # Add reference from another pivot and add the entry after Patient_Id and Instance_Id in the header if not present
    def addRef(self, csv_file, seriesUID, name, pivot, struct):
        self.dict[csv_file + "_-1"][name] = pivot.dict[seriesUID + "_" + struct]["Instance_Id"]
        if name not in self.columns:
            self.columns = self.columns[0:2] + [name] + self.columns[2:]

    def write(self, output):
        self.columns = ["Patient_Id", "Instance_Id"] + self.columns + list(self.dictTag.keys())[:]
        Pivot.write(self, output)
        
    

## Initialize the pivots

In [6]:
pivotAnalysis = PivotDicom(('0x0010', '0x0020'), dictTagAnalysis)
pivotStudy = PivotDicom(('0x0020', '0x000d'), dictTagStudy)
pivotSeries = PivotDicom(('0x0020', '0x000e'), dictTagSeries)
pivotEquipment = PivotDicom(('0x0020', '0x000e'), dictTagEquipment)
pivotInjection = PivotDicom(('0x0020', '0x000e'), dictTagInjection)
pivotROISegmentation = PivotDicom(('0x0008', '0x0018'), dictTagROISegmentation)
pivotCommonImage = PivotDicom(('0x0008', '0x0018'), dictTagCommonImage)
pivotCTImage = PivotDicom(('0x0008', '0x0018'), dictTagCTImage)
pivotDXImage = PivotDicom(('0x0008', '0x0018'), dictTagDXImage)
pivotMRImage = PivotDicom(('0x0008', '0x0018'), dictTagMRImage)
pivotNMImage = PivotDicom(('0x0008', '0x0018'), dictTagNMImage)

pivotRadiotherapyCours = PivotRadiomics()
pivotRadiotherapyPlan = PivotRadiomics()
pivotRadiotherapyVolume = PivotRadiomics()
pivotRadiotherapyDoseVolume = PivotRadiomics()
pivotRadiotherapyTechniqueModality = PivotRadiomics()
pivotRadiotherapyEnergy = PivotRadiomics()

## Fill the pivots
Read all dicoms files and add to the pivots if necessary

In [7]:
#Fill the Analysis pivot from a csv file
analysisFilepath = "OSIRIS_radiotherapy_pivot_Analysis.csv"
csvAnalysisReader = open(analysisFilepath, newline='')
csvAnalysisFile = csv.reader(csvAnalysisReader, delimiter=',', quotechar='|')
for row in csvAnalysisFile:
    if row[0].startswith("Patient"):
        continue
    pivotAnalysis.dict[row[0] + "_-1"] = {
        "Instance_Id": row[1]
    }

In [8]:
for dicom in tqdm(DICOM_files):
    #Read the dicom file
    DICOM_file = pydicom.dcmread(dicom)

    #If the SOP Study UID is not present, I create the entry inside the pivot
    if DICOM_file[('0x0020', '0x000d')].value not in pivotStudy.elements():
        pivotStudy.fill(DICOM_file)
        pivotStudy.dict[DICOM_file[('0x0020', '0x000d')].value + "_-1"]["Study_Location"] = []
        pivotStudy.addRef(DICOM_file, "Analysis_Ref", pivotAnalysis)
    pivotStudy.dict[DICOM_file[('0x0020', '0x000d')].value + "_-1"]["Study_Location"].append(dicom)
    
    #If the SOP Series UID is not present, I create the entry inside the pivots
    if DICOM_file[('0x0020', '0x000e')].value not in pivotSeries.elements():
        pivotEquipment.fill(DICOM_file)
        
        pivotSeries.fill(DICOM_file)
        pivotSeries.addRef(DICOM_file, "Study_Ref", pivotStudy)
        pivotSeries.addRef(DICOM_file, "Equipment_Ref", pivotEquipment)
        
        pivotInjection.fill(DICOM_file)
        pivotInjection.addRef(DICOM_file, "Series_Ref", pivotSeries)

    #For all dicom files, add the element to common image pivot or ROI segmentation pivot
    if DICOM_file[('0x0008', '0x0060')].value in ["CT", "MR", "PT", "DX", "NM"]:
        pivotCommonImage.fill(DICOM_file)
        pivotCommonImage.addRef(DICOM_file, "Series_Ref", pivotSeries)
        
        if DICOM_file[('0x0008', '0x0060')].value == "CT":
            pivotCTImage.fill(DICOM_file)
            pivotCTImage.addRef(DICOM_file, "CommonImage_Ref", pivotCommonImage)
        
        elif DICOM_file[('0x0008', '0x0060')].value == "DX":
            pivotDXImage.fill(DICOM_file)
            pivotDXImage.addRef(DICOM_file, "CommonImage_Ref", pivotCommonImage)
        
        elif DICOM_file[('0x0008', '0x0060')].value == "MR":
            pivotMRImage.fill(DICOM_file)
            pivotMRImage.addRef(DICOM_file, "CommonImage_Ref", pivotCommonImage)
        
        elif DICOM_file[('0x0008', '0x0060')].value == "NM" or DICOM_file[('0x0008', '0x0060')].value == "PT":
            pivotNMImage.fill(DICOM_file)
            pivotNMImage.addRef(DICOM_file, "CommonImage_Ref", pivotCommonImage)
            
    elif DICOM_file[('0x0008', '0x0060')].value == "RTSTRUCT":
        nbStruct = len(DICOM_file[('0x3006', '0x0020')].value)
        for struct in range(nbStruct):
            pivotROISegmentation.fill(DICOM_file, struct)
            pivotROISegmentation.addRef(DICOM_file, "Series_Ref", pivotSeries, struct)



  0%|          | 0/804 [00:00<?, ?it/s]

In [9]:
pivotStudy.columns = pivotStudy.columns[0:6] + ["Study_Location"] + pivotStudy.columns[6:]

pivotStudy.write("OSIRIS_pivot_Study.csv")
pivotSeries.write("OSIRIS_pivot_Series.csv")
pivotEquipment.write("OSIRIS_pivot_Equipment.csv")
pivotInjection.write("OSIRIS_pivot_Injection.csv")
pivotROISegmentation.write("OSIRIS_pivot_ROISegmentation.csv")
pivotCommonImage.write("OSIRIS_pivot_CommonImage.csv")
pivotCTImage.write("OSIRIS_pivot_CTImage.csv")
pivotDXImage.write("OSIRIS_pivot_DXImage.csv")
pivotMRImage.write("OSIRIS_pivot_MRImage.csv")
pivotNMImage.write("OSIRIS_pivot_NMImage.csv")

## Get radiotherapy
Open the .csv files in radiomics folder

In [10]:

pivotRadiotherapyCours.dictTag = {
    "RadiotherapyCours_Type" : "Planned",
    "RadiotherapyCours_Objective" : "CURATIVE",
    "RadiotherapyCours_EndedReason" : "0",
    "RadiotherapyCours_FractionNumber" : "10",
    "RadiotherapyCours_StartDateTime" : "",
    "RadiotherapyCours_EndDateTime" : "",
    }
pivotRadiotherapyPlan.dictTag = {
    "RadiotherapyPlan_Type" : "Planned",
    "RadiotherapyPlan_FractionNumber" : "10",
    "RadiotherapyPlan_StartDateTime" : "",
    "RadiotherapyPlan_EndDateTime" : "",
    "RadiotherapyPlan_ReplanificationReason" : "",
    "RadiotherapyPlan_AlgorithmName" : "Monaco",
    "RadiotherapyPlan_ImageUID" : "1.3.12.2.1107.5.1.7.128048.30000022102408322521400000100",
    "RadiotherapyPlan_RTSSUID" : "2.16.840.1.114337.1.8672.1673611737.1",
    "RadiotherapyPlan_RTDoseUID" : "",
    "RadiotherapyPlan_RTDoseUID" : "2.16.840.1.114337.1.8672.1673611745.0",
    }
pivotRadiotherapyVolume.dictTag = {
    "RadiotherapyVolume_Id" : "SITE",
    "RadiotherapyVolume_Localization" : "anus pelvis ing 2",
    "RadiotherapyVolume_LocalizationCode" : "",
    "RadiotherapyVolume_Description" : "TARGET",
    }
pivotRadiotherapyDoseVolume.dictTag = {
    "RadiotherapyDoseVolume_DosePerFraction" : "3.0",
    "RadiotherapyDoseVolume_NumberFraction" : "10",
    "RadiotherapyDoseVolume_TotalDose" : "30.0",
    }
pivotRadiotherapyTechniqueModality.dictTag = {
    "RadiotherapyTechniqueModality_Technic" : "TREATMENT",
    "RadiotherapyTechniqueModality_Modality" : "PHOTON",
    "RadiotherapyTechniqueModality_MachineName" : "VersaHD",
    }
pivotRadiotherapyEnergy.dictTag = {
    "RadiotherapyEnergy_Quantity" : "6.0",
    "RadiotherapyEnergy_Isotope" : "",
    }

pivotRadiomicsImageFilter.columns = ["Series_Ref"]
for csv_file in csv_files:
    dictTag = {
        "Patient_Id" : dictRadiomicsPatient[os.path.basename(csv_file)],
        "Instance_Id" : dictRadiomicsId[os.path.basename(csv_file)],
    }
    dictTag.update(pivotRadiomicsImageFilter.dictTag)
    pivotRadiomicsImageFilter.dict[csv_file + "_-1"] = dictTag
    pivotRadiomicsImageFilter.addRef(csv_file, dictRadiomicsImageFilter[os.path.basename(csv_file)], "Series_Ref", pivotSeries, "-1")
pivotRadiomicsImageFilter.write("OSIRIS_pivot_RadiomicsImageFilter.csv")

dictRadiomicsCriteria = {
                "radiomics_2022-07-26_osteo.csv": "set_osiris/Osteosarcome/osteo.nii.gz",
                "radiomics_2022-07-26_poumon.csv": "set_osiris/Poumon/13.nii",
                "radiomics_2022-07-26_sein.csv": "set_osiris/Sein/24.nii.gz",
                 }
pivotRadiomicsCriteria.dictTag = {
    "RadiomicsCriteria_SoftwareName" : "PyRadiomics",
    "RadiomicsCriteria_SoftwareVersion" : "v3.0.1",
    "RadiomicsCriteria_ComputationalLocalisationMethod" : "GLOBAL",
    "RadiomicsCriteria_CalculationWindowMatrix" : "",
    "RadiomicsCriteria_UsedImageFilter" : "None",
    "RadiomicsCriteria_UsedImageFilterParameters" : "3x3x3",
    "RadiomicsCriteria_DistanceWeighted" : "None",
    "RadiomicsCriteria_DiscretisationMethod" : "FBS",
    "RadiomicsCriteria_Bound" : "",
    "RadiomicsCriteria_BinSize" : "12",
    "RadiomicsCriteria_LowestIntensity" : "0",
    "RadiomicsCriteria_BiggestIntensity" : "1",
    "RadiomicsCriteria_BoundsRangeOfValueAfterDiscretisation" : "0,1024",
    "RadiomicsCriteria_SpatialResamplingMethod" : "Bicubic spline",
    "RadiomicsCriteria_SpatialResamplingValues" : "[1.0, 2.0, 3.0]",
    "RadiomicsCriteria_TextureMatrixAggregation" : "None",
    }
pivotRadiomicsCriteria.columns = ["ROISegmentation_Ref", "RadiomicsImageFilter_Ref"]
for csv_file in csv_files:
    dictTag = {
        "Patient_Id" : dictRadiomicsPatient[os.path.basename(csv_file)],
        "Instance_Id" : dictRadiomicsId[os.path.basename(csv_file)],
    }
    dictTag.update(pivotRadiomicsCriteria.dictTag)
    pivotRadiomicsCriteria.dict[csv_file + "_-1"] = dictTag
    pivotRadiomicsCriteria.addRef(csv_file, dictRadiomicsCriteria[os.path.basename(csv_file)], "ROISegmentation_Ref", pivotROISegmentation, "1")
    pivotRadiomicsCriteria.addRef(csv_file, csv_file, "RadiomicsImageFilter_Ref", pivotRadiomicsImageFilter, "-1")
pivotRadiomicsCriteria.write("OSIRIS_pivot_RadiomicsCriteria.csv")

# Fill the dict with the current csv file (= add a line in the .csv file)
def fill(pivot, csv_file, key, ref, pid, isbi_radiomicfeature_dict):
        radiomics = csv_file[0]
        values = csv_file[1]
        
        for value, radiomic in enumerate(radiomics):
            if len(radiomic) > 0 and 'Mask-corrected' not in radiomic and 'Mask-original' not in radiomic and 'Image-original' not in radiomic and 'Versions' not in radiomic and 'Configuration' not in radiomic:
                key2 = key + "_" + str(pivot.instanceId)
                pivot.dict[key2] = {}
                pivot.dict[key2]["RadiomicsFeature_Name"] = radiomic
                if radiomic in isbi_radiomicfeature_dict:
                    pivot.dict[key2]["RadiomicsFeature_Id"] = isbi_radiomicfeature_dict[radiomic]
                    if pivot.dict[key2]["RadiomicsFeature_Id"] == "":
                        pivot.dict[key2]["RadiomicsFeature_Id"] = "UMLS:C0439673"
                else:
                    pivot.dict[key2]["RadiomicsFeature_Id"] = "UMLS:C0439673"
                pivot.dict[key2]["RadiomicsFeature_Value"] = values[value]
                pivot.dict[key2]["Instance_Id"] = pivot.instanceId
                pivot.instanceId += 1
                pivot.dict[key2]["RadiomicsCriteria_Ref"] = ref
                pivot.dict[key2]["Patient_Id"] = pid

infile = open('example_RadiomicsFeature.csv', mode='r')
reader = csv.reader(infile)
isbi_radiomicfeature_dict = {rows[3]:rows[5] for rows in reader}

pivotRadiomicsFeature.dictTag = {
    "RadiomicsFeature_Name" : "PyRadiomics",
    "RadiomicsFeature_Value" : "v3.0.1",
    "RadiomicsFeature_Id" : "0",
    }
pivotRadiomicsFeature.columns = ["RadiomicsCriteria_Ref"]
for csv_file in csv_files:
    csvfile = open(csv_file, newline='')
    reader = list(csv.reader(csvfile, delimiter='	', quotechar='|'))
    fill(pivotRadiomicsFeature, reader, csv_file, dictRadiomicsId[os.path.basename(csv_file)], dictRadiomicsPatient[os.path.basename(csv_file)], isbi_radiomicfeature_dict)
pivotRadiomicsFeature.write("OSIRIS_pivot_RadiomicsFeature.csv")
